In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the dataset
df = pd.read_csv('emotion_data.csv')

# Remove rows with invalid sentiments
df = df[df['sentiment'].notna()]  # Remove rows with missing sentiment values
df = df[df['sentiment'].isin(df['sentiment'].unique())]  # Ensure only valid sentiments are included

# Encode labels
df['sentiment'] = df['sentiment'].astype('category')
df['label'] = df['sentiment'].cat.codes

# Print unique labels to verify
print("Unique labels:", df['sentiment'].unique())
print("Encoded labels:", df['label'].unique())

# Split the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['content'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# Check sizes of the datasets
print("Training set size:", len(train_texts))
print("Validation set size:", len(val_texts))

# Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['sentiment'].unique()))

# Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Create dataset class
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = EmotionDataset(train_encodings, train_labels)
val_dataset = EmotionDataset(val_encodings, val_labels)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
try:
    trainer.train()
except RuntimeError as e:
    print(f"Error during training: {e}")

# Save the model
trainer.save_model("emotion_classifier")

# Function to classify sentiments
def classify_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return df['sentiment'].cat.categories[predictions.item()]

# Example usage
if __name__ == "__main__":
    text = "alonzo feels extremely happy!"
    sentiment = classify_sentiment(text)
    print(f"The sentiment classified is: {sentiment}")


Unique labels: ['anger', 'sadness', 'fear', 'joy']
Categories (4, object): ['anger', 'fear', 'joy', 'sadness']
Encoded labels: [0 3 1 2]
Training set size: 6720
Validation set size: 1680


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.002200,0.001418
2,0.000400,0.000254
3,0.000300,0.000174


The sentiment classified is: joy
